In [ ]:
import picamera
from time import sleep
from roboflow import Roboflow
import RPi.GPIO as GPIO

rf = Roboflow(api_key="uFFTlzHjChoByCLZahzo")
project = rf.workspace().project("directional-arrows")
model = project.version(4).model

servoPin = 17

GPIO.setwarnings(False)
GPIO.setmode(GPIO.BCM)
GPIO.setup(servoPin, GPIO.OUT)
pwm = GPIO.PWM(servoPin, 50)
pwm.start(0)

camera = picamera.PiCamera()

# duty cycle
forward_dc = 7.5  
left_dc = 5.5  
right_dc = 9.5
# Function to rotate the servo motor to a specific duty cycle for a given duration
def rotate_servo(duty_cycle, duration):
    pwm.ChangeDutyCycle(duty_cycle)
    time.sleep(duration)

while True:
    camera.resolution = (640,480)
    camera.start_preview()

    sleep(1)

    camera.capture('/home/pi/Desktop/photo.jpg')

    camera.stop_preview()

    direction = model.predict("/home/pi/Desktop/photo.jpg", confidence=40, overlap=30).json()
    arrow_class = direction['predictions'][0]['class']
""" 
    if arrow_class == "right_arrow":
        pwm.start(55)
        sleep(3)

        pwm.stop()
"""

    if arrow_class == 'forward_arrow':
        rotate_servo(forward_dc, 2)  # Rotate forward for 1 second
    elif arrow_class == 'left_arrow':
        rotate_servo(left_dc, 2)
        rotate_servo(forward_dc, 3)# Rotate left for 1 second
    elif direction == 'right_arrow':
        rotate_servo(right_dc, 2)
        rotate_servo(forward_dc, 3)# Rotate right for 1 second
    else:
        print("Invalid direction, try again.")

    sleep(5)